In [1]:
#WARNING TRACK OPTION NEEDS SIGNIFICANT MEMORY
import sys
import json
import time
import os
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import csc_matrix
from scipy.sparse import lil_matrix
from scipy.sparse import hstack
from scipy.sparse import vstack
from scipy.sparse.linalg import svds
import scipy.sparse
import matplotlib.pyplot as plt
import csv
import operator

In [2]:
data_location_desktop = "/home/ubutubiggerbetter/shared/data/spotify_data/data/new_new_formatted_data"
data_location_laptop = "/media/sf_VMDropBox/spotify_data/data/new_formatted_data"
data_location_laptop = "/media/sf_VMDropBox/spotify_data/data/tmp_test"
raw_data_location = os.path.join(data_location_desktop, '..')
data_location = data_location_desktop
save_location = data_location

isdir = os.path.isdir(data_location)
if isdir == False:
    print(data_location, ' does not exist')


In [3]:
Usparse = scipy.sparse.load_npz(os.sep.join((data_location, 'U.npz')))
VTsparse = scipy.sparse.load_npz(os.sep.join((data_location, 'VT.npz')))
playlist_file = os.sep.join((data_location, "tr_playlistname_index_new.csv"))
full_char_mat = scipy.sparse.load_npz(os.sep.join((data_location, 'track_full_char_mat.npz')))
feature_file = os.sep.join((data_location, "track_index_new.csv"))

In [4]:
playlist_indices = sorted([38, 738, 807, 806, 1308, 1563, 1658, 1653, \
                   1965, 4278, 4470, 4553, 5947, 21944, 23637, \
                   29330, 371355, 14613, 377409, 377408, 782899, \
                   804775, 4074])

In [5]:
#RUN DOWN BELOW
def recommender_pl(playlist_indices, Usparse, VTsparse, playlist_file, track_file, \
              full_char_mat, latentfactors_to_use = 3, \
              playlists_to_use = 20, num_recommendations=20):
    
    start_time=  time.time()
    #Convert to useable array:
    
    U = Usparse.toarray()
    VT = VTsparse.toarray()
    #U = Usparse.tolil()
    #VT = VTsparse.tolil()
    #Flip around:
    n = U.shape[1]
    U[:,0:n] = U[:, n-1::-1]
    VT[0:n, :] = VT[n-1::-1,:]

    print('Prepared Information Arrays, elapsed time: ', time.time() - start_time)
    start_time = time.time()
    #Recover Track ingormation
    track_file = os.sep.join((data_location, "track_index_new.csv"))
    track_dict = {}
    track_index_dict = {}
    fobj = open(track_file, "r")
    w = csv.DictReader(fobj)
    for row in w:
        track_dict[row['track_name']] = row['index']
        track_index_dict[row['index']] = row['track_name']
    fobj.close()
    
    #Recover Playlist Information:
    playlist_dict= {}
    pl_index_dict = {}
    fobj = open(playlist_file, "r")
    w = csv.DictReader(fobj)
    for row in w:
        playlist_dict[row['playlist_name']] = row['index']
        pl_index_dict[row['index']] = row['playlist_name']
    fobj.close()
    print('Read in playlist info, elapsed time: ', time.time() - start_time)
    start_time = time.time()
    #Get songs that are in matrix:
    print('Songs in Playlist: ')
    playlist_contents = set()
    for i in playlist_indices:
        name = track_index_dict[str(i)]
        playlist_contents.add(name)
        print(name)

    #Turn playlist_indices into useable matrix:
    plist = csc_matrix((U.shape[0], 1))
    plist[playlist_indices]= 1 
    
    #Map into latent space
    genre_array = plist.transpose() * U
    #Get the top genres:
    top_genres = np.argsort(genre_array)
    top_genres = np.fliplr(top_genres)
    
    #Clear up some stuff: 
    del U
    del Usparse
    del VTsparse
    
    #Use the sorted_indices_pl to map into applicable artists (U)
    recommended_artists = {}
    playlists_used = {}
    MAX_PLAYLISTS = VT.shape[1]
    k = 0
    full_char_mat = full_char_mat.tolil()
    for i in range(0, latentfactors_to_use):
        print(i, ' out of ', latentfactors_to_use)
        #Get the max latent factor index
        latent_factor_i = top_genres[0][i]
        #Get all the playlists' latent factor for this index (corresponds to a given genre)
        latent_factors_v_playlists = VT[latent_factor_i, :]
        #latent_factors_v_playlists = VT[latent_factor_i, :].toarray()
        
        #Get the largest values which will be largest latent factors corresponding
        #to this genre
        asc_similar_playlist_indices = np.argsort(latent_factors_v_playlists)
        similar_playlist_indices = asc_similar_playlist_indices[::-1]
    
        playlists_to_use_loop = playlists_to_use
        k = 0
        while k < playlists_to_use_loop:
            if str(similar_playlist_indices[k]) not in playlists_used:
                print('Similar Playlist is: ', pl_index_dict[str(similar_playlist_indices[k])])
                #Add to the dictionary (so we don't redo)
                playlists_used[str(similar_playlist_indices[k])] = pl_index_dict[str(similar_playlist_indices[k])]
                #get the index; this will be the most similar playlist for the largest latent
                #factor of the query playlist (for now)
                similar_playlist_index = similar_playlist_indices[k]
                #Map the index into original charmat to get similar artists:
                playlists_artists_indices = full_char_mat[:, similar_playlist_index]
            
                #Tally the most similar artists
                non_zero_artist_indices = playlists_artists_indices.nonzero()
                for j in non_zero_artist_indices[0]:
                    nz_index = j
                    if track_index_dict[str(nz_index)] in recommended_artists:
                        recommended_artists[track_index_dict[str(nz_index)]] = recommended_artists[track_index_dict[str(nz_index)]] + 1
                    else:
                        recommended_artists[track_index_dict[str(nz_index)]] = 1
            else:
                if playlists_to_use_loop + 1 < MAX_PLAYLISTS:
                    playlists_to_use_loop = playlists_to_use_loop + 1
            k = k+1
    
    sorted_dict = sorted(recommended_artists.items(), key=operator.itemgetter(1), reverse=True)

    top_recommended_artists = []
    

    #Just go through and compare to queried playlist, as long as item is not
    #already in, then add it (note: with artists this may be dicey....)
    i = 0
    for item, value in sorted_dict:
        if item not in playlist_contents:
            if i >= num_recommendations:
                break
            top_recommended_artists.append(item)
            i = i+1

    return top_recommended_artists
    
 

In [6]:
latent_factors_used = 1
playlists_referenced = 20
desired_recommendations = 10
start_time = time.time()
#Will give recommendations given playlist_indices
recommendations = recommender_pl(playlist_indices, Usparse, VTsparse, playlist_file, \
                                 feature_file, full_char_mat, latent_factors_used, playlists_referenced, \
                                 desired_recommendations)

print('Elapsed: ', time.time() - start_time)
print('Recommendations: ', recommendations)


Prepared Information Arrays, elapsed time:  4.61146354675293
Read in playlist info, elapsed time:  12.443139791488647
Songs in Playlist: 
Mr. Brightside
Tubthumping
Semi-Charmed Life
Jumper - 1998 Edit
Little Lion Man
Roses
Wonderwall
Creep - Acoustic
Hey There Delilah
Welcome To The Jungle
Shoop (Re-Recorded) [Remastered]
Drops of Jupiter
Ophelia
Kryptonite
First Date
Drunken Lullabies
A Sky Full Of Stars - Radio Edit
Pumpin Blood - The Chainsmokers Remix
In My Youth
Rolling In the Deep (Acoustic Tribute to Adele)
Sweet Child O' mine (Acoustic)
I Gotta Feeling (Blackeyed Peas)
I'm Shipping Up to Boston


/home/ubutubiggerbetter/anaconda3/envs/w_pyspark/lib/python3.9/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


0  out of  1
Similar Playlist is:  Alt_420
Similar Playlist is:  alt/rock
Similar Playlist is:  Alternative_978
Similar Playlist is:  alt_192
Similar Playlist is:  Alternative/Pop
Similar Playlist is:  Alternative _49
Similar Playlist is:  Not Rap_11
Similar Playlist is:  rockish_19
Similar Playlist is:  All Songs_17
Similar Playlist is:  Boner Jams_1
Similar Playlist is:  alternative_487
Similar Playlist is:  Hallelujah_21
Similar Playlist is:  Alt_389
Similar Playlist is:  Alternative_863
Similar Playlist is:  Alternative Rock_163
Similar Playlist is:  Nicks Playlist
Similar Playlist is:  Alternative_384
Similar Playlist is:  chill_2656
Similar Playlist is:  alternative_293
Similar Playlist is:  Alternative_611
Elapsed:  169.53533577919006
Recommendations:  ['Float On', 'Use Somebody', "Ain't No Rest for the Wicked - Original Version", 'Home', 'Seven Nation Army', 'Little Talks', 'Take Me Out', 'When You Were Young', 'Do I Wanna Know?', 'Under The Bridge']
